In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

In [ ]:

# Evaluation Metrics
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import r2_score

In [ ]:
SEED = 1234

In [ ]:
# Load original DataSet and contact_df
from google.colab import drive
drive.mount('/content/drive') #,force_remount=Truimport os 
cwd = os.getcwd()
dataset_dir = os.path.join(cwd, 'DMTMDataset')
# train_df=pd.read_csv(os.path.join(dataset_dir,'train.csv'))
dataset_direction='/content/drive/MyDrive/'
# !unzip ('/content/drive/MyDrive/')
train_df=open(os.path.join(dataset_direction,'train.csv'))
original_df=pd.read_csv((train_df))

PCA_origin=open(os.path.join(dataset_direction,'PCA_origin.csv'))

from numpy import genfromtxt
PCA_arr = genfromtxt(PCA_origin, delimiter=',')
concat_df=pd.DataFrame(PCA_arr)

Mounted at /content/drive


In [ ]:
concat_df.describe

<bound method NDFrame.describe of               0         1         2   ...        29        30        31
0      -0.638963 -0.723192  1.073850  ...  0.025030 -0.178321  0.071160
1      -0.643231 -0.908548  1.280059  ...  0.014163  0.119573  0.014776
2      -0.630286 -1.024725  1.449132  ...  0.015206  0.299571 -0.017298
3      -0.615162 -0.894503  1.380044  ...  0.020561  0.489645 -0.063044
4      -0.622776 -1.300381  1.845811  ...  0.016869  0.930811 -0.140841
...          ...       ...       ...  ...       ...       ...       ...
621295 -0.677910 -0.951450  1.305120  ...  0.004824  0.198475 -0.059389
621296 -0.692814 -0.798725  1.111425  ...  0.004765  0.217437 -0.073611
621297 -0.693233 -0.784259  1.072286  ...  0.010330  0.111350 -0.045168
621298 -0.772449 -0.395100  0.378322  ... -0.002375 -0.109610 -0.021795
621299 -0.745592 -0.460563  0.531000  ... -0.000575 -0.053972 -0.031844

[621300 rows x 32 columns]>

In [ ]:
# train_data = concat_df.copy()
# train_data.head()

In [ ]:
#shuffle trainDf
# train_data.sample(frac=1)

In [ ]:
# X=train_data.drop(['aircon_sum_target_next14d'], axis=1)
# y=train_data['aircon_sum_target_next14d']
X = concat_df.copy()
y = original_df['aircon_sum_target_next14d']


In [ ]:
from collections import Counter
cn = Counter(y) 
estimator = cn[0]/cn[1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [ ]:
# ind_params = {'max_depth': 10, 'n_estimators': 1000, 'learning_rate' : 0.1}

In [ ]:
# model=xgb.XGBClassifier(**ind_params)
# model.fit(X_train,y_train)
# y_pred=model.predict(X_test)

[13:20:06] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [ ]:
# model=xgb.XGBRFClassifier(**ind_params)
# model.fit(X_train,y_train)
# y_pred=model.predict(X_test)

[14:53:26] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot
# grid search
model=xgb.XGBClassifier(scale_pos_weight=estimator)
n_estimators = [200,500]
learning_rate = [0.01,0.6]
param_grid = dict(learning_rate=learning_rate, n_estimators=n_estimators)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))


Best: -0.105597 using {'learning_rate': 0.6, 'n_estimators': 500}
-0.550557 (0.000583) with: {'learning_rate': 0.01, 'n_estimators': 200}
-0.501368 (0.001204) with: {'learning_rate': 0.01, 'n_estimators': 500}
-0.220881 (0.002028) with: {'learning_rate': 0.6, 'n_estimators': 200}
-0.105597 (0.001593) with: {'learning_rate': 0.6, 'n_estimators': 500}


In [ ]:
best_model = grid_result.best_estimator_
y_pred = best_model.predict(X_test)

In [ ]:
print("R2:",r2_score(y_test, y_pred))
print("\tAccuracy (Test) %.3f"%precision_score(y_test, y_pred))
print("\t      Precision %.3f"%precision_score(y_test, y_pred))
print("\t      Recall    %.3f"%recall_score(y_test, y_pred))
print("\t      F1        %.3f"%f1_score(y_test, y_pred))
print("\n")

R2: 0.0653235817798904
	Accuracy (Test) 0.894
	      Precision 0.894
	      Recall    0.080
	      F1        0.147


